In [1]:
!pip install keras-rectified-adam
!pip install -U git+https://github.com/qubvel/efficientnet

  Created wheel for keras-rectified-adam: filename=keras_rectified_adam-0.17.0-cp36-none-any.whl size=14781 sha256=cd29489ece42e2ae7c94a6fae912fb32a726b9b39fb9e02598c5b7d89e8c24ca
  Stored in directory: /root/.cache/pip/wheels/7b/01/27/3a934e1a5644f5b93c720422a6ef97034ea78a21ba71cfb549
Successfully built keras-rectified-adam
  Cloning https://github.com/qubvel/efficientnet to /tmp/pip-req-build-0z9skdth
  Running command git clone -q https://github.com/qubvel/efficientnet /tmp/pip-req-build-0z9skdth
  Created wheel for efficientnet: filename=efficientnet-1.1.0-cp36-none-any.whl size=18328 sha256=d2e17636a8e27fd6677d21881a23d81673ca6265007e780ace9375e4a814c0b9
  Stored in directory: /tmp/pip-ephem-wheel-cache-xccgnjqg/wheels/64/60/2e/30ebaa76ed1626e86bfb0cc0579b737fdb7d9ff8cb9522663a
Successfully built efficientnet


In [2]:
import os, glob, random, cv2
import keras
import multiprocessing

import numpy as np
import pandas as pd
import keras.backend as K
import matplotlib.pyplot as plt

from copy import deepcopy

from sklearn.metrics import precision_recall_curve, auc
from sklearn.model_selection import train_test_split

from keras.optimizers import Adam
from keras.callbacks import Callback
from keras.applications.densenet import DenseNet201
from keras.layers import Dense, Flatten
from keras.models import Model, load_model
from keras.utils import Sequence
from keras_radam import RAdam

from albumentations import Compose, VerticalFlip, HorizontalFlip, Rotate, GridDistortion
from IPython.display import Image
from tqdm import tqdm_notebook as tqdm
from numpy.random import seed
from tensorflow import set_random_seed

seed(10)
set_random_seed(10)
%matplotlib inline

Using TensorFlow backend.


In [3]:
test_imgs_dir = '../input/understanding_cloud_organization/test_images/'
train_imgs_dir = '../input/understanding_cloud_organization/train_images/'
num_cores = multiprocessing.cpu_count()

# Data Generators

## One hot encoding

In [4]:
train_df = pd.read_csv('../input/understanding_cloud_organization/train.csv')
train_df.head()

,Image_Label,EncodedPixels
0,0011165.jpg_Fish,264918 937 266318 937 267718 937 269118 937 27...
1,0011165.jpg_Flower,1355565 1002 1356965 1002 1358365 1002 1359765...
2,0011165.jpg_Gravel,NaN
3,0011165.jpg_Sugar,NaN
4,002be4f.jpg_Fish,233813 878 235213 878 236613 878 238010 881 23...


In [5]:
train_df = train_df[~train_df['EncodedPixels'].isnull()]
train_df['Image'] = train_df['Image_Label'].map(lambda x: x.split('_')[0])
train_df['Class'] = train_df['Image_Label'].map(lambda x: x.split('_')[1])

classes = train_df['Class'].unique()

train_df = train_df.groupby('Image')['Class'].agg(set).reset_index()

for class_name in classes:
    train_df[class_name] = train_df['Class'].map(lambda x: 1 if class_name in x else 0)

train_df.head()

,Image,Class,Fish,Flower,Sugar,Gravel
0,0011165.jpg,"{Fish, Flower}",1,1,0,0
1,002be4f.jpg,"{Fish, Sugar, Flower}",1,1,1,0
2,0031ae9.jpg,"{Fish, Sugar, Flower}",1,1,1,0
3,0035239.jpg,"{Gravel, Flower}",0,1,0,1
4,003994e.jpg,"{Fish, Sugar, Gravel}",1,0,1,1


In [6]:
# Dictionary for fast access
img_2_ohe_vector = {img:vec for img, vec in zip(train_df['Image'], train_df.iloc[:, 2:].values)}

In [7]:
# train-val split
train_imgs, val_imgs = train_test_split(train_df['Image'].values, 
                                        test_size = 0.2, 
                                        stratify = train_df['Class'].map(lambda x: str(sorted(list(x)))), 
                                        random_state = 2019)

In [8]:
class DataGenenerator(Sequence):
    def __init__(self, images_list = None, folder_imgs = train_imgs_dir, 
                 batch_size = 32, shuffle = True, augmentation = None,
                 resized_height = 260, resized_width = 260, num_channels = 3):
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.augmentation = augmentation
        
        if images_list is None:
            self.images_list = os.listdir(folder_imgs)
        else:
            self.images_list = deepcopy(images_list)
        
        
        self.folder_imgs = folder_imgs
        self.len = len(self.images_list) // self.batch_size
        self.resized_height = resized_height
        self.resized_width = resized_width
        self.num_channels = num_channels
        self.num_classes = 4
        self.is_test = not 'train' in folder_imgs
        
        if not shuffle and not self.is_test:
            self.labels = [img_2_ohe_vector[img] for img in self.images_list[:self.len * self.batch_size]]

    def __len__(self):
        return self.len
    
    def on_epoch_start(self):
        if self.shuffle:
            random.shuffle(self.images_list)

    def __getitem__(self, idx):
        current_batch = self.images_list[idx * self.batch_size: (idx + 1) * self.batch_size]
        X = np.empty((self.batch_size, self.resized_height, self.resized_width, self.num_channels))
        y = np.empty((self.batch_size, self.num_classes))

        for i, image_name in enumerate(current_batch):
            path = os.path.join(self.folder_imgs, image_name)
            img = cv2.resize(cv2.imread(path), (self.resized_height, self.resized_width)).astype(np.float32)
            
            if not self.augmentation is None:
                augmented = self.augmentation(image=img)
                img = augmented['image']
            
            X[i, :, :, :] = img/255.0
            
            if not self.is_test:
                y[i, :] = img_2_ohe_vector[image_name]
        
        return X, y

    def get_labels(self):
        if self.shuffle:
            images_current = self.images_list[:self.len * self.batch_size]
            labels = [img_2_ohe_vector[img] for img in images_current]
        else:
            labels = self.labels
        return np.array(labels)

In [9]:
albumentations_train = Compose([ VerticalFlip(), HorizontalFlip(), Rotate(limit=20), GridDistortion() ], p = 1)

In [10]:
# generator instances
data_generator_train = DataGenenerator(train_imgs, augmentation = albumentations_train)
data_generator_train_eval = DataGenenerator(train_imgs, shuffle = False)
data_generator_val = DataGenenerator(val_imgs, shuffle = False)

In [11]:
class PrAucCallback(Callback):
    def __init__(self, data_generator, num_workers = num_cores, 
                 early_stopping_patience = 5, 
                 plateau_patience = 3, reduction_rate = 0.5,
                 stage = 'train', checkpoints_path = 'checkpoints/'):
        super(Callback, self).__init__()
        self.data_generator = data_generator
        self.num_workers = num_workers
        self.class_names = ['Fish', 'Flower', 'Sugar', 'Gravel']
        self.history = [[] for _ in range(len(self.class_names) + 1)] # to store per each class and also mean PR AUC
        self.early_stopping_patience = early_stopping_patience
        self.plateau_patience = plateau_patience
        self.reduction_rate = reduction_rate
        self.stage = stage
        self.best_pr_auc = -float('inf')
        
        if not os.path.exists(checkpoints_path):
            os.makedirs(checkpoints_path)
        
        self.checkpoints_path = checkpoints_path
        
    def compute_pr_auc(self, y_true, y_pred):
        pr_auc_mean = 0
        for class_i in range(len(self.class_names)):
            precision, recall, _ = precision_recall_curve(y_true[:, class_i], y_pred[:, class_i])
            pr_auc = auc(recall, precision)
            pr_auc_mean += pr_auc/len(self.class_names)
            print(f"PR-AUC {self.class_names[class_i]}, {self.stage}: {pr_auc:.3f}")
            self.history[class_i].append(pr_auc)        
        print(f"\nmean, {self.stage}: {pr_auc_mean:.3f}")
        self.history[-1].append(pr_auc_mean)
        return pr_auc_mean
              
    def is_patience_lost(self, patience):
        if len(self.history[-1]) > patience:
            best_performance = max(self.history[-1][-(patience + 1):-1])
            return best_performance == self.history[-1][-(patience + 1)] and best_performance >= self.history[-1][-1]    
              
    def early_stopping_check(self, pr_auc_mean):
        if self.is_patience_lost(self.early_stopping_patience):
            self.model.stop_training = True    
              
    def model_checkpoint(self, pr_auc_mean, epoch):
        if pr_auc_mean > self.best_pr_auc:
            
            # remove previous checkpoints to save space
            for checkpoint in glob.glob(os.path.join(self.checkpoints_path, 'classifier_epoch_*')):
                os.remove(checkpoint)
            self.best_pr_auc = pr_auc_mean
            self.model.save(os.path.join(self.checkpoints_path, f'classifier_epoch_{epoch}_val_pr_auc_{pr_auc_mean}.h5'))              
            print(f"\nSaved new checkpoint")
              
    def reduce_lr_on_plateau(self):
        if self.is_patience_lost(self.plateau_patience):
            new_lr = float(keras.backend.get_value(self.model.optimizer.lr)) * self.reduction_rate
            keras.backend.set_value(self.model.optimizer.lr, new_lr)
            print(f"\nReduced learning rate to {new_lr}.")
        
    def on_epoch_end(self, epoch, logs={}):
        y_pred = self.model.predict_generator(self.data_generator, workers=self.num_workers)
        y_true = self.data_generator.get_labels()
        
        # estimate AUC under precision recall curve for each class
        pr_auc_mean = self.compute_pr_auc(y_true, y_pred)
              
        if self.stage == 'val':
            self.early_stopping_check(pr_auc_mean)
            self.model_checkpoint(pr_auc_mean, epoch)
            self.reduce_lr_on_plateau()            
        
    def get_pr_auc_history(self):
        return self.history

In [12]:
train_metric_callback = PrAucCallback(data_generator_train_eval)
val_callback = PrAucCallback(data_generator_val, stage='val')

In [13]:
# CLASSIFIER

from keras.losses import binary_crossentropy
def dice_coef(y_true, y_pred, smooth=1):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

def dice_loss(y_true, y_pred):
    smooth = 1.
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = y_true_f * y_pred_f
    score = (2. * K.sum(intersection) + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)
    return 1. - score

def bce_dice_loss(y_true, y_pred):
    return binary_crossentropy(y_true, y_pred) + dice_loss(y_true, y_pred)

In [14]:
import efficientnet.keras as efn 
def get_model():
    K.clear_session()
    base_model =  efn.EfficientNetB0(weights='imagenet', include_top=False, pooling='avg', input_shape=(260, 260, 3))
#     base_model =  efn.EfficientNetB1(weights='imagenet', include_top=False, pooling='avg', input_shape=(260, 260, 3))
#     base_model =  efn.EfficientNetB1(weights='imagenet', include_top=False, pooling='avg', input_shape=(260, 260, 3))
#     base_model =  efn.EfficientNetB2(weights='imagenet', include_top=False, pooling='avg', input_shape=(260, 260, 3))
#     base_model =  efn.EfficientNetB3(weights='imagenet', include_top=False, pooling='avg', input_shape=(260, 260, 3))
#     base_model =  efn.EfficientNetB4(weights='imagenet', include_top=False, pooling='avg', input_shape=(260, 260, 3))
#     base_model =  efn.EfficientNetB5(weights='imagenet', include_top=False, pooling='avg', input_shape=(260, 260, 3))
    x = base_model.output
    y_pred = Dense(4, activation='sigmoid')(x)
    return Model(inputs = base_model.input, outputs = y_pred)

model = get_model()

16809984/16804768 [==============================] - 0s 0us/step


In [ ]:
# Initial tuning of the added fully-connected layer
for base_layer in model.layers[:-3]:
    base_layer.trainable = False
    
model.compile(optimizer = RAdam(warmup_proportion = 0.1, min_lr = 1e-5),  
                              loss = 'categorical_crossentropy', 
                              metrics = ['accuracy'])

history_0 = model.fit_generator(generator = data_generator_train,
                              validation_data = data_generator_val,
                              epochs = 20,
                              callbacks = [train_metric_callback, val_callback],
                              workers = num_cores,
                              verbose = 1)

In [ ]:
# Fine-tuning the whole model
for base_layer in model.layers[:-3]:
    base_layer.trainable = True
    
model.compile(optimizer = RAdam(warmup_proportion = 0.1, min_lr = 1e-5),  
                              loss = 'categorical_crossentropy', 
                              metrics = ['accuracy'])
history_1 = model.fit_generator(generator = data_generator_train,
                              validation_data = data_generator_val,
                              epochs = 20,
                              callbacks = [train_metric_callback, val_callback],
                              workers = num_cores,
                              verbose = 1,
                              initial_epoch = 1)

In [ ]:
fs = 10
fnt1 = 12
fnt2 = 15
pr_auc_history_train = train_metric_callback.get_pr_auc_history()
pr_auc_history_val = val_callback.get_pr_auc_history()

plt.figure(figsize = (fs, fs))
plot(plt, pr_auc_history_train[-1])
plot(plt, pr_auc_history_val[-1])

plt.xlabel('Epoch', fontsize = fnt1)
plt.ylabel('Mean PR-AUC', fontsize = fnt1)
plt.legend(['Train', 'Validation'])
plt.title('Training & validation PR-AUC', fontsize = fnt2)
plt.savefig('pr_auc_hist.png')

In [ ]:
plt.figure(figsize = (fs, fs))
plot_with_dots(plt, history_0.history['loss']+history_1.history['loss'])
plot_with_dots(plt, history_0.history['val_loss']+history_1.history['val_loss'])

plt.xlabel('Epoch', fontsize = fnt1)
plt.ylabel('Binary Crossentropy', fontsize = fnt1)
plt.legend(['Train', 'Validation'])
plt.title('Training & Validation Loss', fontsize = fnt2)
plt.savefig('loss_hist.png')

In [ ]:
# select  post processing threshold
class_names = ['Fish', 'Flower', 'Sugar', 'Gravel']
def get_threshold_for_recall(y_true, y_pred, class_i, recall_threshold = 0.94, precision_threshold = 0.90, plot = False):
    precision, recall, thresholds = precision_recall_curve(y_true[:, class_i], y_pred[:, class_i])
    i = len(thresholds) - 1
    best_recall_threshold = None
    while best_recall_threshold is None:
        next_threshold = thresholds[i]
        next_recall = recall[i]
        if next_recall >= recall_threshold:
            best_recall_threshold = next_threshold
        i -= 1
        
    # consice, even though unnecessary passing through all the values
    best_precision_threshold = [thres for prec, thres in zip(precision, thresholds) if prec >= precision_threshold][0]
    
    if plot:
        plt.figure(figsize = (fs, fs))
        plt.step(recall, precision, color='g', alpha=0.3, where='post')
        plt.fill_between(recall, precision, alpha=0.3, color='b')
        plt.axhline(y=precision[i + 1])
        recall_for_prec_thres = [rec for rec, thres in zip(recall, thresholds) if thres == best_precision_threshold][0]
        plt.axvline(x=recall_for_prec_thres, color='r')
        plt.xlabel('Recall')
        plt.ylabel('Precision')
        plt.ylim([0.0, 1.05])
        plt.xlim([0.0, 1.0])
        plt.legend(['PR curve', 
                    f'Precision {precision[i + 1]: .2f} corresponding to selected recall threshold',
                    f'Recall {recall_for_prec_thres: .2f} corresponding to selected precision threshold'])
        plt.title(f'Precision-Recall curve for Class {class_names[class_i]}')
    return best_recall_threshold, best_precision_threshold

y_pred = model.predict_generator(data_generator_val, workers=num_cores)
y_true = data_generator_val.get_labels()
recall_thresholds = dict()
precision_thresholds = dict()
for i, class_name in tqdm(enumerate(class_names)):
    recall_thresholds[class_name], precision_thresholds[class_name] = get_threshold_for_recall(y_true, y_pred, i, plot=True)

In [21]:
# SUBMISSION
# prediction of cloud classes
data_generator_test = DataGenenerator(folder_imgs=test_imgs_dir, shuffle=False)
y_pred_test = model.predict_generator(data_generator_test, workers=num_cores)

# Estimating set of images without masks
image_labels_empty = set()
for i, (img, predictions) in enumerate(zip(os.listdir(test_imgs_dir), y_pred_test)):
    for class_i, class_name in enumerate(class_names):
        if predictions[class_i] < recall_thresholds[class_name]:
            image_labels_empty.add(f'{img}_{class_name}')

# Segmentation results:            
submission = pd.read_csv('../input/densenet201cloudy/densenet201.csv')
submission.head()

predictions_nonempty = set(submission.loc[~submission['EncodedPixels'].isnull(), 'Image_Label'].values)

print(f'{len(image_labels_empty.intersection(predictions_nonempty))} masks would be removed')

#removing masks
submission.loc[submission['Image_Label'].isin(image_labels_empty), 'EncodedPixels'] = np.nan
submission.to_csv('submission.csv', index=None)